## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [0]:
%tensorflow_version 2.x
import tensorflow as tf

In [0]:
print (tf.__version__)

2.0.0


In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv("//content/drive/My Drive/Great Learning/prices.csv", delimiter=",")

### Check all columns in the dataset

In [0]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data.drop(columns=['date','symbol'],axis=1,inplace=True)

In [0]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
data = data[:1000]

In [0]:
x = data.iloc[:,0:4]

In [0]:
y = data.iloc[:,4:]/1000000
y

,volume
0,2.1636
1,2.3864
2,2.4895
3,2.0063
4,1.4086
...,...
995,2.1332
996,1.9824
997,37.1528
998,6.5686


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.20,random_state=3)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
import numpy as np
X_train =np.array(X_train).astype('float32')
X_test = np.array(X_test).astype('float32')
Y_train =np.array(Y_train).astype('float32')
Y_test = np.array(Y_test).astype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer

transformer = Normalizer()
X_train = transformer.fit_transform(X_train)
X_test = transformer.transform(X_test)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
W = tf.zeros(shape=(4, 1))
b = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
@tf.function
def prediction(X_train, W, b):
    y_pred = tf.add(tf.matmul(X_train, W), b)
    return y_pred

3.Loss (Cost) Function [Mean square error]

In [0]:
@tf.function
def loss(Y_train, y_predicted):
    diff = Y_train - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
@tf.function
def train(X_train, Y_train, w, b, learning_rate=0.01):
    
    # Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        t.watch([w,b])
        current_prediction = prediction(X_train, w, b)
        current_loss = loss(Y_train, current_prediction)
    
    # Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    # Update Weights and Bias
    w = w - learning_rate * dw
    b = b - learning_rate * db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [0]:
for i in range(100):    
    W, b = train(X_train, Y_train, W, b)
    print('Current Training Loss on iteration', i, ' -- ', loss(Y_train, prediction(X_train, w, b)).numpy())

Current Training Loss on iteration 0  --  198.8946
Current Training Loss on iteration 1  --  197.85219
Current Training Loss on iteration 2  --  196.8709
Current Training Loss on iteration 3  --  195.9467
Current Training Loss on iteration 4  --  195.07599
Current Training Loss on iteration 5  --  194.25531
Current Training Loss on iteration 6  --  193.4814
Current Training Loss on iteration 7  --  192.7514
Current Training Loss on iteration 8  --  192.06245
Current Training Loss on iteration 9  --  191.41205
Current Training Loss on iteration 10  --  190.79778
Current Training Loss on iteration 11  --  190.2173
Current Training Loss on iteration 12  --  189.66867
Current Training Loss on iteration 13  --  189.14989
Current Training Loss on iteration 14  --  188.6591
Current Training Loss on iteration 15  --  188.19469
Current Training Loss on iteration 16  --  187.75507
Current Training Loss on iteration 17  --  187.33871
Current Training Loss on iteration 18  --  186.94421
Current Tr

In [0]:
for i in range(100):    
    W, b = train(X_train, Y_train, W, b)
    if i%5 == 0:
      print('Current Training Loss on iteration : ', i,' -- ', loss(Y_train, prediction(X_train, w, b)).numpy())
    

Current Training Loss on iteration :  0  --  179.24971
Current Training Loss on iteration :  5  --  179.20726
Current Training Loss on iteration :  10  --  179.1727
Current Training Loss on iteration :  15  --  179.14465
Current Training Loss on iteration :  20  --  179.12173
Current Training Loss on iteration :  25  --  179.10309
Current Training Loss on iteration :  30  --  179.08794
Current Training Loss on iteration :  35  --  179.07555
Current Training Loss on iteration :  40  --  179.06548
Current Training Loss on iteration :  45  --  179.05727
Current Training Loss on iteration :  50  --  179.05055
Current Training Loss on iteration :  55  --  179.04507
Current Training Loss on iteration :  60  --  179.04059
Current Training Loss on iteration :  65  --  179.03699
Current Training Loss on iteration :  70  --  179.03398
Current Training Loss on iteration :  75  --  179.0316
Current Training Loss on iteration :  80  --  179.02959
Current Training Loss on iteration :  85  --  179.02

### Get the shapes and values of W and b

In [0]:
print ('shape of W : ', W)

shape of W :  tf.Tensor(
[[1.2862533]
 [1.3554296]
 [1.2863494]
 [1.3583086]], shape=(4, 1), dtype=float32)


In [0]:
print ('shape of W : ', b)

shape of W :  tf.Tensor([2.6439803], shape=(1,), dtype=float32)


### Model Prediction on 1st Examples in Test Dataset

In [0]:
pred=prediction(X_test,W,b)
pred

<tf.Tensor: id=5311, shape=(200, 1), dtype=float32, numpy=
array([[5.287775 ],
       [5.2875805],
       [5.287441 ],
       [5.2872868],
       [5.287381 ],
       [5.288076 ],
       [5.287963 ],
       [5.287582 ],
       [5.2873015],
       [5.287275 ],
       [5.287551 ],
       [5.2878036],
       [5.287967 ],
       [5.2871723],
       [5.2877884],
       [5.287678 ],
       [5.287855 ],
       [5.28751  ],
       [5.287779 ],
       [5.287405 ],
       [5.287567 ],
       [5.287188 ],
       [5.2871556],
       [5.2872806],
       [5.2874107],
       [5.287141 ],
       [5.287696 ],
       [5.28675  ],
       [5.286813 ],
       [5.287193 ],
       [5.2879047],
       [5.287507 ],
       [5.287929 ],
       [5.287789 ],
       [5.2871504],
       [5.2876463],
       [5.2872586],
       [5.2872095],
       [5.287731 ],
       [5.2872295],
       [5.287771 ],
       [5.2864695],
       [5.287985 ],
       [5.287569 ],
       [5.28723  ],
       [5.2872   ],
       [5.287592 ],
 

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
iris = pd.read_csv("//content/drive/My Drive/Great Learning/11_Iris.csv")

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
# import labelencoder
from sklearn.preprocessing import LabelEncoder
# instantiate labelencoder object
le = LabelEncoder()
# apply le on categorical feature columns
iris.Species = le.fit_transform(iris.Species)

In [0]:
y_iris = tf.keras.utils.to_categorical(iris.Species, num_classes=3)

### Splitting the data into feature set and target set

In [0]:
X_iris = iris.iloc[:,1:5]
#y_iris = iris.Species
y_iris.shape

(150, 3)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
# Initialize Sequential model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(4, input_dim=4, kernel_initializer='normal', activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

In [0]:

# Create optimizer with non-default learning rate
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)

# Compile the model
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(X_iris, y_iris, test_size=0.20,random_state=3)

### Model Training 

In [0]:
model.fit(trainX, trainY, validation_data=(testX, testY), epochs=30)

Train on 120 samples, validate on 30 samples
Epoch 1/30
120/120 [==============================] - 0s 3ms/sample - loss: 0.8206 - accuracy: 0.6667 - val_loss: 0.7208 - val_accuracy: 0.6667
Epoch 2/30
120/120 [==============================] - 0s 74us/sample - loss: 0.7098 - accuracy: 0.6667 - val_loss: 0.6654 - val_accuracy: 0.6667
Epoch 3/30
120/120 [==============================] - 0s 79us/sample - loss: 0.6571 - accuracy: 0.6667 - val_loss: 0.6201 - val_accuracy: 0.6667
Epoch 4/30
120/120 [==============================] - 0s 83us/sample - loss: 0.6136 - accuracy: 0.6667 - val_loss: 0.5817 - val_accuracy: 0.6667
Epoch 5/30
120/120 [==============================] - 0s 75us/sample - loss: 0.5762 - accuracy: 0.6667 - val_loss: 0.5451 - val_accuracy: 0.6667
Epoch 6/30
120/120 [==============================] - 0s 83us/sample - loss: 0.5410 - accuracy: 0.6667 - val_loss: 0.5150 - val_accuracy: 0.6667
Epoch 7/30
120/120 [==============================] - 0s 81us/sample - loss: 0.5083 - 

### Model Prediction

In [0]:
model.predict(testX)

array([[2.5517231e-01, 5.3898168e-01, 2.0584604e-01],
       [2.5517231e-01, 5.3898168e-01, 2.0584604e-01],
       [2.5517231e-01, 5.3898168e-01, 2.0584604e-01],
       [2.8921449e-01, 5.1432610e-01, 1.9645940e-01],
       [2.5517231e-01, 5.3898168e-01, 2.0584604e-01],
       [9.9900573e-01, 7.1808870e-04, 2.7618386e-04],
       [9.4745058e-01, 3.7989806e-02, 1.4559634e-02],
       [2.5517231e-01, 5.3898168e-01, 2.0584604e-01],
       [9.9229008e-01, 5.5711335e-03, 2.1388407e-03],
       [9.5597643e-01, 3.1824697e-02, 1.2198844e-02],
       [9.3701917e-01, 4.5533158e-02, 1.7447678e-02],
       [2.5517231e-01, 5.3898168e-01, 2.0584604e-01],
       [9.4211721e-01, 4.1846506e-02, 1.6036272e-02],
       [9.5342988e-01, 3.3666067e-02, 1.2903996e-02],
       [9.9340880e-01, 4.7625378e-03, 1.8286635e-03],
       [2.5517231e-01, 5.3898168e-01, 2.0584604e-01],
       [9.8388267e-01, 1.1648337e-02, 4.4690319e-03],
       [9.9657357e-01, 2.4754000e-03, 9.5102459e-04],
       [9.8845702e-01, 8.341

### Save the Model

In [0]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?